In [1]:
import pandas as pd
import numpy as np
import os
print(os.listdir('.'))

['.git', 'A1_ARIMA.ipynb', 'data', 'ProcessAthleteTable.ipynb', 'ProcessCountryTable.ipynb', 'ProcessedData', 'ProcessMedalTable.ipynb', 'README.md']


In [2]:
file_medals = 'ProcessedData/Medals.csv'
df_medals = pd.read_csv(
    file_medals,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
print(df_medals)

      Rank  NOC  Gold  Silver  Bronze  Total  Year
0        1  USA    11       7       2     20  1896
1        2  GRE    10      18      19     47  1896
2        3  GER     6       5       2     13  1896
3        4  FRA     5       4       2     11  1896
4        5  GBR     2       3       2      7  1896
...    ...  ...   ...     ...     ...    ...   ...
1424    84  PER     0       0       1      1  2024
1425    84  QAT     0       0       1      1  2024
1426    84  SGP     0       0       1      1  2024
1427    84  SVK     0       0       1      1  2024
1428    84  ZAM     0       0       1      1  2024

[1429 rows x 7 columns]


In [3]:
# 获取所有不同的国家代码 NOC, 和所有年份 year
countries = df_medals['NOC'].unique().tolist()
print(countries)
years = df_medals['Year'].unique().tolist()
print(years)

['USA', 'GRE', 'GER', 'FRA', 'GBR', 'HUN', 'AUT', 'AUS', 'DEN', 'SUI', 'BEL', 'ITA', 'CUB', 'CAN', 'ESP', 'LUX', 'NED', 'NOR', 'IND', 'BOH', 'SWE', 'ANZ', 'RUS', 'FIN', 'RSA', 'EST', 'BRA', 'JPN', 'TCH', 'NZL', 'YUG', 'ARG', 'URU', 'POL', 'HAI', 'POR', 'ROU', 'EGY', 'IRL', 'CHI', 'PHI', 'MEX', 'LAT', 'TUR', 'JAM', 'PER', 'SRI', 'TTO', 'KOR', 'PAN', 'IRI', 'PUR', 'URS', 'LBN', 'BUL', 'VEN', 'ISL', 'PAK', 'BAH', 'ETH', 'TPE', 'GHA', 'MAR', 'SGP', 'IRQ', 'TUN', 'KEN', 'NGR', 'GDR', 'FRG', 'MGL', 'UGA', 'CMR', 'PRK', 'COL', 'NIG', 'BER', 'THA', 'ZIM', 'TAN', 'GUY', 'CHN', 'CIV', 'SYR', 'ALG', 'DOM', 'ZAM', 'SUR', 'AHO', 'CRC', 'INA', 'ISV', 'SEN', 'DJI', 'EUN', 'LTU', 'NAM', 'CRO', 'ISR', 'SLO', 'MAS', 'QAT', 'UKR', 'CZE', 'KAZ', 'BLR', 'SVK', 'ARM', 'BDI', 'ECU', 'HKG', 'MDA', 'UZB', 'AZE', 'TGA', 'GEO', 'MOZ', 'KSA', 'VIE', 'BAR', 'KGZ', 'KUW', 'MKD', 'UAE', 'SCG', 'PAR', 'ERI', 'SRB', 'TJK', 'SAM', 'SUD', 'AFG', 'MRI', 'TOG', 'BRN', 'GRN', 'BOT', 'CYP', 'GAB', 'GUA', 'MNE', 'FIJ', 'JOR'

In [4]:
# 按国家从 DataFrame 中提取各个年份的奖牌数量
result = {}

for country in countries:
    # print(country)
    country_data = df_medals[df_medals['NOC'] == country]
    gold_list = []
    silver_list = []
    bronze_list = []
    total_list = []
    
    for year in years:
        # print(year)
        year_data = country_data[country_data['Year'] == year]
        
        gold = int(year_data['Gold']) if not year_data.empty else 0
        silver = int(year_data['Silver']) if not year_data.empty else 0
        bronze = int(year_data['Bronze']) if not year_data.empty else 0
        total = int(year_data['Total']) if not year_data.empty else 0
        gold_list.append(gold)
        silver_list.append(silver)
        bronze_list.append(bronze)
        total_list.append(total)
    
    result[country] = {
        'Gold': gold_list,
        'Silver': silver_list,
        'Bronze': bronze_list,
        'Total': total_list
    }

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_37004\4116312352.py:16: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  gold = int(year_data['Gold']) if not year_data.empty else 0
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_37004\4116312352.py:17: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  silver = int(year_data['Silver']) if not year_data.empty else 0
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_37004\4116312352.py:18: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  bronze = int(year_data['Bronze']) if not year_data.empty else 0
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_37004\4116312352.py:19: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use

In [5]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

In [6]:
for country, data in result.items():
    print(f"Country: {country}")
    print("Gold: ", data['Gold'])
    print("Silver: ", data['Silver'])
    print("Bronze: ", data['Bronze'])
    print("Total: ", data['Total'])
    
    model = ARIMA(data['Gold'], order=(2,1,1)) #目前参数不一定最优，如何比较？
    model_fit = model.fit()
    forecast1 = model_fit.forecast(steps=1) #预测下一届
    model = ARIMA(data['Silver'], order=(2,1,1))
    model_fit = model.fit()
    forecast2 = model_fit.forecast(steps=1)
    model = ARIMA(data['Bronze'], order=(2,1,1))
    model_fit = model.fit()
    forecast3 = model_fit.forecast(steps=1)
    model = ARIMA(data['Total'], order=(2,1,1))
    model_fit = model.fit()
    forecast4 = model_fit.forecast(steps=1)
    print(f"Predicted Medals for next Olympics: {round(forecast1[0])}, {round(forecast2[0])}, {round(forecast3[0])}, {round(forecast4[0])}")

Country: USA
Gold:  [11, 19, 76, 23, 26, 41, 45, 22, 44, 24, 38, 40, 32, 34, 36, 45, 33, 34, 0, 83, 36, 37, 44, 37, 36, 36, 48, 46, 39, 40]
Silver:  [7, 14, 78, 12, 19, 27, 27, 18, 36, 21, 27, 19, 25, 21, 26, 28, 31, 35, 0, 61, 31, 34, 32, 24, 39, 39, 26, 37, 41, 44]
Bronze:  [2, 15, 77, 12, 19, 27, 27, 16, 30, 12, 19, 17, 17, 16, 28, 34, 30, 25, 0, 30, 27, 37, 25, 32, 26, 37, 30, 38, 33, 42]
Total:  [20, 48, 231, 47, 64, 95, 99, 56, 110, 57, 84, 76, 74, 71, 90, 107, 94, 94, 0, 174, 94, 108, 101, 93, 101, 112, 104, 121, 113, 126]
Predicted Medals for next Olympics: 40, 29, 32, 100
Country: GRE
Gold:  [10, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 4, 4, 6, 0, 0, 3, 2, 1]
Silver:  [18, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 4, 6, 6, 2, 0, 1, 1, 1]
Bronze:  [19, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 3, 4, 1, 2, 2, 1, 6]
Total:  [47, 0, 2, 4, 2, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 0, 3, 2, 1, 2, 8, 13, 16, 3, 2, 6, 4,

d:\Jupyter\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
d:\Jupyter\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


Predicted Medals for next Olympics: 1, 1, 2, 7
Country: GER
Gold:  [6, 4, 4, 3, 5, 0, 0, 10, 5, 38, 0, 0, 6, 12, 10, 0, 0, 0, 0, 0, 0, 33, 20, 13, 13, 16, 11, 17, 10, 12]
Silver:  [5, 3, 5, 5, 13, 0, 0, 7, 12, 31, 0, 7, 13, 19, 22, 0, 0, 0, 0, 0, 0, 21, 18, 17, 16, 11, 20, 10, 11, 13]
Bronze:  [2, 2, 6, 5, 7, 0, 0, 14, 7, 32, 0, 17, 7, 11, 18, 0, 0, 0, 0, 0, 0, 28, 27, 26, 20, 14, 13, 15, 16, 8]
Total:  [13, 9, 15, 13, 25, 0, 0, 31, 24, 101, 0, 24, 26, 42, 50, 0, 0, 0, 0, 0, 0, 82, 65, 56, 49, 41, 44, 42, 37, 33]
Predicted Medals for next Olympics: 10, 11, 11, 30
Country: FRA
Gold:  [5, 27, 0, 5, 7, 9, 13, 6, 11, 7, 10, 6, 4, 0, 1, 7, 2, 2, 6, 5, 6, 8, 15, 13, 11, 7, 11, 10, 10, 16]
Silver:  [4, 39, 1, 5, 4, 19, 15, 10, 5, 6, 6, 6, 4, 2, 8, 3, 4, 3, 5, 7, 4, 5, 7, 14, 9, 16, 11, 18, 12, 26]
Bronze:  [2, 37, 0, 9, 3, 13, 10, 5, 4, 6, 13, 6, 6, 3, 6, 5, 7, 4, 3, 16, 6, 16, 15, 11, 13, 20, 13, 14, 11, 22]
Total:  [11, 103, 1, 19, 14, 41, 38, 21, 20, 19, 29, 18, 14, 5, 15, 15, 13, 9, 14, 2

d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Predicted Medals for next Olympics: 0, 0, 0, 0
Country: NED
Gold:  [0, 0, 0, 0, 0, 4, 4, 6, 2, 6, 5, 0, 0, 0, 2, 3, 3, 0, 0, 5, 2, 2, 4, 12, 4, 7, 6, 8, 10, 15]
Silver:  [0, 2, 0, 0, 0, 2, 1, 9, 5, 4, 2, 5, 0, 1, 4, 3, 1, 2, 1, 2, 2, 6, 5, 9, 9, 5, 6, 7, 12, 7]
Bronze:  [0, 3, 0, 2, 3, 5, 5, 4, 1, 7, 9, 0, 0, 2, 4, 1, 1, 3, 2, 6, 5, 7, 10, 4, 9, 4, 8, 4, 14, 12]
Total:  [0, 5, 0, 2, 3, 11, 10, 19, 8, 17, 16, 5, 0, 3, 10, 7, 5, 5, 3, 13, 9, 15, 19, 25, 22, 16, 20, 19, 36, 34]
Predicted Medals for next Olympics: 12, 8, 10, 36
Country: NOR
Gold:  [0, 0, 2, 2, 3, 13, 5, 1, 0, 1, 1, 3, 1, 1, 0, 1, 2, 1, 0, 0, 2, 2, 2, 4, 5, 3, 2, 0, 4, 4]
Silver:  [0, 2, 0, 3, 2, 9, 2, 2, 0, 3, 3, 2, 0, 0, 0, 1, 1, 1, 0, 1, 3, 4, 2, 3, 0, 5, 1, 0, 2, 1]
Bronze:  [0, 3, 0, 3, 5, 9, 3, 1, 0, 2, 3, 0, 2, 0, 0, 0, 1, 0, 0, 2, 0, 1, 3, 3, 1, 1, 1, 4, 2, 3]
Total:  [0, 5, 2, 8, 10, 31, 10, 4, 0, 6, 7, 5, 3, 1, 0, 2, 4, 2, 0, 3, 5, 7, 7, 10, 6, 9, 4, 4, 8, 8]
Predicted Medals for next Olympics: 3, 2, 2, 7
Country:

...

d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Predicted Medals for next Olympics: 0, 1, 0, 1
Country: BUR
Gold:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Silver:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Bronze:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Total:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Predicted Medals for next Olympics: 0, 0, 1, 1
Country: LCA
Gold:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Silver:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Bronze:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Total:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
Predicted Medals for next Olympics: 1, 1, 0, 2
Country: DMA
Gold:  [0, 0, 0, 0, 0, 0, 

d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn

Predicted Medals for next Olympics: 1, 0, 0, 1
Country: ALB
Gold:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Silver:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Bronze:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
Total:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
Predicted Medals for next Olympics: 0, 0, 2, 2
Country: CPV
Gold:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Silver:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Bronze:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Total:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Predicted Medals for next Olympics: 0, 0, 1, 1
Country: EOR
Gold:  [0, 0, 0, 0, 0, 0, 

d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Jupyter\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
import warnings
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# 预测2024年的来作为评估
N = len(result['USA']['Gold'])
ms = ['Gold', 'Silver', 'Bronze', 'Total']
# 只对某一代码块忽略警告
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for m in ms:
        mse = 0
        rmse = 0
        mae = 0
        mape = 0
        r2 = 0
        tests = []
        predicts = []
        for country, data in result.items():
            try:
                train, test = data[m][:N-1], data[m][N-1]
                tests.append(test)
                
                model = ARIMA(train, order=(2, 1, 1))
                model_fit = model.fit()
                prediction = model_fit.forecast(steps=1)
                prediction = np.round(prediction[0]).astype(int)
                predicts.append(prediction)
            except Exception as e:
                pass
        
        print(f"tests: {tests}")
        print(f"preds: {predicts}")
        
        # 计算评估指标
        mse = mean_squared_error(tests, predicts) #均方误差
        rmse = np.sqrt(mse) #均方根误差
        mae = mean_absolute_error(tests, predicts) #平均绝对误差
        # mape = np.mean(np.abs((tests-predicts)/tests))*100 #平均绝对百分比误差，有0！
        r2 = r2_score(tests, predicts) #[0,1]，越靠近1，效果越好
            
        # 打印评估结果
        print(f'Mean Squared Error (MSE): {mse}')
        print(f'Root Mean Squared Error (RMSE): {rmse}')  #TODO：可作为误差范围？
        print(f'Mean Absolute Error (MAE): {mae}')
        # print(f'Mean Absolute Percentage Error (MAPE): {mape}%')
        print(f'R-squared (R²): {r2}')